In [74]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [76]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [78]:
target = "efs"
features = [col for col in train.columns if col not in ["efs", "efs_time", "ID"]]  # Exclude target and ID columns

X_train = train[features]
y_train = train[target]
X_test = test[features]

In [80]:
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=[object]).columns.tolist()

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")), 
    ("scaler", StandardScaler())  
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),  
    ("encoder", OneHotEncoder(handle_unknown="ignore"))  
])

In [82]:
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])# Step 6: Combine Transformers into a Preprocessor
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

In [84]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LogisticRegression()) 
])

In [86]:
pipeline.fit(X_train, y_train)

C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['hla_match_c_high',
                                                   'hla_high_res_8',
                                                   'hla_low_res_6',
                                                   'hla_high_res_6',
                                                   'hla_high_res_10',
                                                   'hla_match_dqb1_high',
                                                   'hla_nmdp_6',
                                                   'hla_match_c_low',
                                                   'hla_match_drb1_low',
                                                   'hla_match_dqb1_low',
                                                   'year...
                                                   'renal_issue', 'pulm_severe',
                                                   'prim_disease_hct',
                                                   'cmv_status',
                                                   'tce_imm_match', 'rituximab',
                                                   'prod_type',
                                                   'cyto_score_detail',
                                                   'conditioning_intensity',
                                                   'ethnicity', 'obesity',
                                                   'mrd_hct', 'in_vivo_tcd',
                                                   'tce_match',
                                                   'hepatic_severe',
                                                   'prior_tumor',
                                                   'peptic_ulcer', 'gvhd_proph',
                                                   'rheum_issue', 'sex_match',
                                                   'race_group', 'hepatic_mild', ...])])),
                ('model', LogisticRegression())])

In [98]:
predictions = pipeline.predict_proba(X_test)[:, 1]
predictions

array([0.15256618, 0.5512715 , 0.09297956])

In [90]:
sample_submission["efs"] = predictions
sample_submission.to_csv("submission.csv", index=False)

In [92]:
print("✅ Submission file created successfully: submission.csv")

✅ Submission file created successfully: submission.csv


In [94]:
from lifelines.utils import concordance_index

# Compute C-index (higher is better)
c_index = concordance_index(y_true=valid_efs_time, y_pred=predicted_efs_time, event_observed=valid_efs)
print(f"Concordance Index: {c_index:.4f}")

ModuleNotFoundError: No module named 'lifelines'

In [96]:
from sksurv.metrics import brier_score
from sksurv.functions import StepFunction

# Convert survival times into cumulative risk predictions
times = np.linspace(0, max(valid_efs_time), 100)
risk_preds = np.array([StepFunction(time, pred).predict(times) for time, pred in zip(valid_efs_time, predicted_probs)])

# Compute Brier Score
brier = brier_score(valid_efs, valid_efs_time, risk_preds, times)
print(f"Brier Score: {brier.mean():.4f}")


ModuleNotFoundError: No module named 'sksurv'